# 1. Importing Necessary Packages

In [15]:
import numpy as np
import cv2

# 2. Function to resize the image

In [16]:
def imageResize(img, perc=0.5):
    img_width = int(img.shape[1] * perc)
    img_height = int(img.shape[0] * perc)
    resizedImage = cv2.resize(img, (img_width, img_height), interpolation=cv2.INTER_AREA)
    return resizedImage

# 3. Function to Calculate Dark Channel

In [17]:
def calculateDarkChannel(img,sz=15):
    blue, green, red = cv2.split(img)
    darkCh = red - cv2.max(blue, green)
    kernelVal = cv2.getStructuringElement(cv2.MORPH_RECT, (sz, sz))
    darkChannel = cv2.erode(darkCh, kernelVal)
    return darkChannel

# 4. Function to Calculate air light of an image

In [18]:
def calculateAirlight(img, dc):
    [height, width] = img.shape[:2]
    imageSize = height * width
    imgVector = img.reshape(imageSize, 3)
    darkChvec = dc.reshape(imageSize)
    indices = darkChvec.argsort()
    return imgVector[indices[0]]

# 5. Function to Calculate Transmission Estimate

In [19]:
def calculateTransmissionEstimate(darkCh):
    transmissionEstimate = darkCh + (1 - np.max(darkCh))
    return transmissionEstimate

# 6. Function to refine the filter

In [20]:
def refineFilter(img, p, r, eps):
    mean_I = cv2.boxFilter(img, cv2.CV_64F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F, (r, r))
    mean_Ip = cv2.boxFilter(img * p, cv2.CV_64F, (r, r))
    cov_Ip = mean_Ip - mean_I * mean_p

    mean_II = cv2.boxFilter(img * img, cv2.CV_64F, (r, r))
    var_I = mean_II - mean_I * mean_I

    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I

    mean_a = cv2.boxFilter(a, cv2.CV_64F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_64F, (r, r))

    refinedOutput = mean_a * img + mean_b
    return refinedOutput

# 7. Refine the transmission map created

In [21]:
def refineTransmission(img, et):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray) / 255
    r = 60
    eps = 0.0001
    refinedT = refineFilter(gray, et, r, eps)
    return refinedT

# 8. Normalize the intensities of the pixels in the image

In [22]:
def normalizeImage(img):
    img = img - img.min()
    img = img / img.max() * 255
    img = np.uint8(img)
    return img

# 9. Recover the final image after transmission map and air light

In [23]:
def recover(img, t, A):
    finalRes = np.empty(img.shape, img.dtype)
    for index in range(0, 3):
        finalRes[:, :, index] = (img[:, :, index] - A[index]) / t + A[index]
    return finalRes

# 10. Enter the Input File name, read the Image and resize it

In [24]:
filename = input("Enter the name of the file with extension: ")
scale = 1

src = cv2.imread(f'images/input_images/{filename}')
src = imageResize(src, scale)

img = src.astype('float64') / 255

Enter the name of the file with extension: img1.png


# 11. Dehaze the given image and store it in the output directory

In [25]:
while True: 
    ksize = 2
    dc = calculateDarkChannel(img,ksize)
    te = calculateTransmissionEstimate(dc)
    tr = refineTransmission(src, te)
    A = calculateAirlight(img, tr)
    result = recover(img, tr, A)
    result = normalizeImage(result)
    
    cv2.namedWindow("Input Image",cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Input Image", 800, 600)
    cv2.imshow("Input Image", src)
    
    cv2.namedWindow("Result",cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Result", 800, 600)
    cv2.imshow("Result", result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

cv2.imwrite(f'images/output_images/{filename}', result)

True